In [13]:
import numpy as np 
import pandas as pd 
import json # library to handle JSON files

from geopy.geocoders import Nominatim 

import requests
from pandas.io.json import json_normalize 

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

from sklearn.cluster import KMeans


import folium # map rendering library

print('Libraries imported.')

Libraries imported.


### 1) We scrape the Wikipedia page using Beautiful Soup.

In [2]:
import urllib.request, urllib.parse,urllib.error
from bs4 import BeautifulSoup

req = requests.get("https://en.wikipedia.org/w/index.php?title=List_of_postal_codes_of_Canada:_M&oldid=945633050")
soup = BeautifulSoup(req.content,'lxml')
table = soup.find_all('table')[0]
df_list = pd.read_html(str(table))
print(df_list)

[    Postcode           Borough          Neighbourhood
0        M1A      Not assigned           Not assigned
1        M2A      Not assigned           Not assigned
2        M3A        North York              Parkwoods
3        M4A        North York       Victoria Village
4        M5A  Downtown Toronto           Harbourfront
..       ...               ...                    ...
282      M8Z         Etobicoke              Mimico NW
283      M8Z         Etobicoke     The Queensway West
284      M8Z         Etobicoke  Royal York South West
285      M8Z         Etobicoke         South of Bloor
286      M9Z      Not assigned           Not assigned

[287 rows x 3 columns]]


### 2) Creating a dataframe containing all data & change the name of dataframe columns:

In [3]:
# Create a dataframe containing all data (using concat function):
df = pd.concat(df_list)

#After, we rename Postcode column:
df=df.rename(columns={"Postcode": "Postal Code","Neighbourhood": "Neighborhood"})

df.head()

,Postal Code,Borough,Neighborhood
0,M1A,Not assigned,Not assigned
1,M2A,Not assigned,Not assigned
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Harbourfront


### 3) We eliminate 'Not assigned' postal codes rows: 


In [19]:
df=df[df.Borough !='Not assigned'].reset_index()
df.head()

,index,Postal Code,Borough,Neighborhood
0,2,M3A,North York,Parkwoods
1,3,M4A,North York,Victoria Village
2,4,M5A,Downtown Toronto,Harbourfront
3,5,M6A,North York,Lawrence Heights
4,6,M6A,North York,Lawrence Manor


### 4) Grouping neighborhoods

In [20]:
#First, we apply groupby command in the data frame:
df2=df.groupby('Postal Code')['Neighborhood'].apply(lambda Neighborhood: ', '.join(Neighborhood)).reset_index()
df2.head()

,Postal Code,Neighborhood
0,M1B,"Rouge, Malvern"
1,M1C,"Highland Creek, Rouge Hill, Port Union"
2,M1E,"Guildwood, Morningside, West Hill"
3,M1G,Woburn
4,M1H,Cedarbrae


In [23]:
#We incorporate the borough column to the dataframe:

boroughs=[]
for i in range(0,len(df2)):
    for j in range(0,len(df)):
        if df2.loc[i,'Postal Code']==df.loc[j,'Postal Code']:
            boroughs.append(df.loc[j,'Borough'])
            break
        else:
            continue

df2['Borough'] = boroughs

#Finally, we sort the columns:
df2 = df2[['Postal Code','Borough','Neighborhood']]
df2.head()

,Postal Code,Borough,Neighborhood
0,M1B,Scarborough,"Rouge, Malvern"
1,M1C,Scarborough,"Highland Creek, Rouge Hill, Port Union"
2,M1E,Scarborough,"Guildwood, Morningside, West Hill"
3,M1G,Scarborough,Woburn
4,M1H,Scarborough,Cedarbrae


In [24]:
df2.shape

(103, 3)